# Ch4 exercises

## 1. What Linear Regression training algorithm can you use if you have a training set with millions of features?

If the whole training set fits in our memory we can use Batch, MB, and SGD. If not, we could use MB or SGD. Main disadvantage of SGD is training time as it goes thru every sample. With MB we go in small random batches.

## 2. Suppose the features in your training set have very different scales. What algorithms might suffer from this, and how? What can you do about it?

To fix this we should scale the data, for example using StdScaler from sklearn. Algorithms that suffer from this might be algorithms that use more than 2 features to train their models. We can name Polynomial Regression for example, or any other
algo. that uses more than 2 features. This affects the algorithm by creatin weird shapes on the hyperplane of error calculation. If scales are the same, then the minimun can be found easier.

## 3. Can Gradient Descent get stuck in a local minimum when training a Logistic Regression model?

Yes, as with any other model because of the nature of GD.

WRONG.

Logit. Regression cost function is convex, so it has no local minima to fail in.

## 4. Do all Gradient Descent algorithms lead to the same model provided you let them run long enough?

Apart from hyperparameters, all GD algorithms should lead to the same model which is finding the global minimun. The main problem is finding a local minimun and not being able to scape from it.
This depends on the learning rate mainly. A good learning rate will be able to scape from local minima and approach the global without any problem.

PARTLY WRONG.

SGD and MB will keep "jumping" around global minimun. We can stop this with a precission threshold.

## 5. Suppose you use Batch Gradient Descent and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely going on? How can you fix this?

If the validation set RMSE reaches a minimun and then goes up, while the test set RMSE is still going down, means that we are overfitting our model.

## 6. Is it a good idea to stop Mini-batch Gradient Descent immediately when the validation error goes up?

Not immediately. We can wait a certain amount of epochs just in case it starts going down again. We can set a certain amount of epochs to wait.

## 7. Which Gradient Descent algorithm (among those we discussed) will reach the vicinity of the optimal solution the fastest? Which will actually converge? How can you make the others converge as well?

The fastest one is Batch, as it takes into account the whole dataset. Batch will also converge, the others wont, but will stay near the global minimun.
To make then converge we can set a threshold difference on the error between previous epoch and the actual one. If the error is not changing, then we can finish.

WRONG.

SGD is the fastest because of its random nature.

## 8. Suppose you are using Polynomial Regression. You plot the learning curves and you notice that there is a large gap between the training error and the validation error. What is happening? What are three ways to solve this?

This means that the model is overfitting, as it is training better much better on training than on validation. To fix this we can use more samples, maybe scale the data, and use less degrees on the model.

## 9. Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter α or reduce it?

NO ANSWER.

If both errors are high, the model is probably underfitting. We can reduce alpha to smooth the model and reduce the bias.

## 10. Why would you want to use:

### a. Ridge Regression instead of plain Linear Regression (i.e., without any regularization)?

It is always preferable to have a bit a bit of regularization, just in case. Maybe on ideal data.

### b. Lasso instead of Ridge Regression?

If we are not certain of the weight of some features. Lasso can drop down weights on useless features.

### c. Elastic Net instead of Lasso?

Elastic Net is prefered because Lasso can fail when some features are strongly correlated.

## 11. Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two Logistic Regression classifiers or one Softmax Regression classifier?

We can directly use Softmax as it combines multiple binary classifiers.

WRONG.

Those two classes are not mutually excluyent. We should have two Logit. Regressors.


## 12. Implement Batch Gradient Descent with early stopping for Softmax Regression (without using Scikit-Learn).